In [2]:
import math
from pomegranate import *

In [24]:
import csv

with open('INTC.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    aberturas = [] # lista para guardar os valores de abertura
    fechamentos = [] # lista para os fechamentos
    
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are: {", ".join(row)}')
            line_count += 1
        else:
            #print(row)
            aberturas.append(row[1])
            fechamentos.append(row[4])
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are: Date, Open, High, Low, Close, Adj Close, Volume
Processed 252 lines.


In [26]:
#print(aberturas)

In [5]:
#print(fechamentos)

In [27]:
# calculo da probabilidade de subir
count_subidas = 0
count_descidas = 0
for i in range(0, line_count-1): #contagem do número de dias que valor da ação subiu
    if aberturas[i] < fechamentos[i]:
        count_subidas += 1
    else:                        #contagem do número de descidas
        count_descidas += 1
        
prob_subir = count_subidas/(line_count-1) #P(Subir)
prob_descer = count_descidas/(line_count-1) #P(Descer)

# probabilidade de fechar um dia qualquer em alta ou queda
print(prob_subir)
print(prob_descer)

0.5378486055776892
0.46215139442231074


In [29]:
estado_hoje = DiscreteDistribution( { 'prob_subir': prob_subir, 'prob_descer': prob_descer} )

In [31]:
#print(estado_hoje)

In [32]:
# Como não se sabe o que ocorrerá, assume-se igual probabilidade
estado_amanha = DiscreteDistribution( { 'prob_subir': 0.5, 'prob_descer': 0.5 } )

In [34]:
#print(estado_amanha)

In [36]:
#O bloco abaixo realiza o cálculo das probabilidades condicionais dado que:
#prob1 = P(SubirAmanhã|SubiuHoje)
#prob2 = P(SubirAmanhã|DesceuHoje)
#prob3 = P(DescerAmanhã|DesceuHoje)
#prob4 = P(DescerAmanhã|SubiuHoje)

prob1 = 0.5 * prob_subir
prob2 = 0.5 * prob_descer
prob3 = 1 - prob1
prob4 = 1 - prob2

print(prob1)
print(prob2)
print(prob3)
print(prob4)

0.2689243027888446
0.23107569721115537
0.7310756972111554
0.7689243027888446


In [40]:
# Tabela de probabilidades condicionais
previsao_amanha = ConditionalProbabilityTable(
    [
        [ 'prob_subir', 'prob_subir', prob1 ],
        [ 'prob_subir', 'prob_descer', prob2 ],
        [ 'prob_descer', 'prob_descer', prob3 ],
        [ 'prob_descer', 'prob_subir', prob4 ],
    ],
    [estado_hoje, estado_amanha]    
)

In [46]:
s1 = State( estado_hoje, name="estado_hoje" )
s2 = State( estado_amanha, name="estado_amanha" )
s3 = State( previsao_amanha, name="previsao_amanha" )
 
#Building the Bayesian Network
network = BayesianNetwork( "Teste..." )
network.add_states(s1, s2, s3)
network.add_edge(s1, s2) # estado_hoje é pai de amanhã
network.add_edge(s2, s3)
network.bake()

KeyError: ('prob_subir', 'prob_subir', 'prob_subir')

In [47]:
#print(network)

In [48]:
beliefs = network.predict_proba({'estado_amanha' : 'prob_subir', 'previsao_amanha' : 'prob_subir'})
print("n".join( "{}t{}".format( state.name, str(belief) ) for state, belief in zip( network.states, beliefs )))

AttributeError: 'NoneType' object has no attribute 'log_probability'